# 第2回講義　演習　Word2Vec(Skipgram, CBOW)の実装

Word2Vecとは2013年に当時GoogleにいたTomas Mikolovらによって発表された論文に端を発する、単語の分散表現を学習するためのアルゴリズムの総称です。

NLPのタスクにおいて入力は単語で構成されていますが、単語を単語のまま扱っていてはコンピュータにとってはただの文字の羅列でしかなく、翻訳や文書分類などの複雑なNLPのタスクを解くことは困難です。

そのため単語やフレーズ、文などの意味を持つ要素をいかに有用な意味表現ないし特徴量に変換できるかがタスクのパフォーマンスに直結します。

Word2Vecは単語から有用な特徴量（ベクトル）を学習することができる最も有名なツールの１つです。Word2Vecを使うと意味的に似たような単語は似たようなベクトルに変換することができ、さらには得られた単語のベクトルが加法性を持つという特徴があります。例えば、"Japan" - "Tokyo" ≒ "France" - "Paris"など単語を使った足し算・引き算を擬似的に行えます。

下図のグラフは、Word2Vecのアルゴリズムの1つであるSkipgramで単語のベクトルを獲得した後にPCAを使って次元圧縮をしたのちにいくつかの国名とその首都のベクトルを可視化したものです。教師なし学習であるにも関わらず、このような国と首都の関係まで学習できてしまうのがWord2Vecが有名になった要因の一つでもあります。

<img src=../image/country-city.png width=500>
<div style="text-align: center;">
出典:[Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
</div>

Word2vecの学習モデルは大きく分けて2種類あり、CBOWとSkipgramと呼ばれています。

CBOWは周囲の単語を元に単語を予測するタスク、Skipgramは周囲の単語を予測するタスクを解けるような単語のベクトルを学習します。

これらはともに単語は周囲の単語によって特徴づけられるという"分布仮説"(distributional hypothesis)に基づいていると捉えることができます。

この演習ではCBOWとSkipgramをPyTorchを使って実装し、実際に日本語のデータセットで学習します。(そのため日本語の前処理も扱います。)さらに、効率的な学習方法であるNegative Samplingも実装します。

In [1]:
import MeCab
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 必要であれば変更してください
import os
os.chdir('/root/userspace/chap2/materials')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1. 前処理

### 1.1. 形態素解析

文などを分割して、言語で意味を持つ最小単位である形態素の列に変換し、その品詞を推定することを形態素解析と呼びます。

In [2]:
tagger = MeCab.Tagger("-Ochasen")
node = tagger.parse("坊主が屏風に上手に坊主の絵を描いた")
print(node)

坊主	ボウズ	坊主	名詞-一般		
が	ガ	が	助詞-格助詞-一般		
屏風	ビョウブ	屏風	名詞-一般		
に	ニ	に	助詞-格助詞-一般		
上手	ジョウズ	上手	名詞-形容動詞語幹		
に	ニ	に	助詞-副詞化		
坊主	ボウズ	坊主	名詞-一般		
の	ノ	の	助詞-連体化		
絵	エ	絵	名詞-一般		
を	ヲ	を	助詞-格助詞-一般		
描い	エガイ	描く	動詞-自立	五段・カ行イ音便	連用タ接続
た	タ	た	助動詞	特殊・タ	基本形
EOS



### 1.2. MeCabを用いて文を形態素に分割

先ほどの形態素解析の結果を用いて、日本語の文を形態素に分割する関数を定義します。

In [3]:
def tokenize(sentence):
    """日本語の文を形態素の列に分割する関数

    :param sentence: str, 日本語の文
    :return tokenized_sentence: list of str, 形態素のリスト
    """
    node = tagger.parse(sentence)
    node = node.split("\n")
    tokenized_sentence = []
    for i in range(len(node)):
        feature = node[i].split("\t")
        if feature[0] == "EOS":
            # 文が終わったら終了
            break
        # 分割された形態素を追加
        tokenized_sentence.append(feature[0])
    return tokenized_sentence

In [4]:
tokenize("坊主が屏風に上手に坊主の絵を描いた")

['坊主', 'が', '屏風', 'に', '上手', 'に', '坊主', 'の', '絵', 'を', '描い', 'た']

### 1.3. データ読み込み

夏目漱石の『こころ』(../data/kokoro.txt)を題材にします。

In [5]:
# 頭から5行を表示
! head -5 ../data/kokoro.txt

私はその人を常に先生と呼んでいた。
だからここでもただ先生と書くだけで本名は打ち明けない。
これは世間を憚かる遠慮というよりも、その方が私にとって自然だからである。
私はその人の記憶を呼び起すごとに、すぐ「先生」といいたくなる。
筆を執っても心持は同じ事である。


In [6]:
def load_data(path):
    """『こころ』を読み込むための関数

    :param path: str, 『こころ』のパス
    :return text: list of list of str, 各文がトークナイズされた『こころ』
    """
    text = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            line = tokenize(line)
            text.append(line)
    return text

In [7]:
text = load_data("../data/kokoro.txt")

In [8]:
# 分割された結果の例
print(text[0])

['私', 'は', 'その', '人', 'を', '常に', '先生', 'と', '呼ん', 'で', 'い', 'た', '。']


### 1.4. 辞書構築
コーパス（文書データ）に登場する単語にユニークなIDを振るために辞書を構築します。

全ての単語を辞書に登録するとしばしば語彙数が膨大(1万〜100万)になりメモリが足りなくなることがあるので、

単語の出現頻度で足切りして辞書のサイズを制限するということをします。

NLPではほぼ必ず必要になる手順です。

In [9]:
# 特殊なトークンとそのIDは事前に定義しておきます。
PAD_TOKEN = '<PAD>' # あとで説明するpaddingに使います
UNK_TOKEN = '<UNK>' # 辞書にない単語は全てこのUNKトークンに置き換えます。(UNKの由来はunkownです)
PAD = 0 # <PAD>のID
UNK = 1 # <UNK>のID

In [10]:
# 辞書の初期化
word2id = {
    PAD_TOKEN: PAD,
    UNK_TOKEN: UNK,
}

# 辞書に含める単語の最低出現回数
# 今回はコーパスのサイズが小さいので、全ての単語を辞書に含めることにします
MIN_COUNT = 1

In [11]:
class Vocab(object):
    """語彙を管理するためのクラス"""
    def __init__(self, word2id={}):
        """
        :param word2id: 単語(str)をインデックス(int)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    

    def build_vocab(self, sentences, min_count=1):
        """コーパスから語彙の辞書を構築するメソッド

        :param sentences: list of list of str, コーパス
        :param min_count: int, 辞書に含める単語の最小出現回数
        """
        # 各単語の出現回数をカウントする辞書を作成します
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                # dict.get(key, 0)はdictにkeyがあればdict[key]を、なければ0を返すメソッドです
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加えます
        # 出現回数の高い単語から順にword2idに追加していきます
        # 出現回数に-1をかけた値でsortすることで出現回数の降順になるようにしています
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word

        # 語彙に含まれる単語の出現回数を保持します（あとで使います）
        self.raw_vocab = {w: word_counter[w] for w in self.word2id.keys() if w in word_counter}

In [12]:
vocab = Vocab(word2id=word2id)
vocab.build_vocab(text, min_count=MIN_COUNT)
print("語彙数:", len(vocab.word2id))

語彙数: 6659


### 1.5. 単語のID化

In [13]:
def sentence_to_ids(vocab, sen):
    """
    単語のリストをIDのリストに変換する関数

    :param vocab: class `Vocab` object
    :param sen: list of str, 文を分かち書きして得られた単語のリスト
    :return out: list of int, 単語IDのリスト
    """
    out = [vocab.word2id.get(word, UNK) for word in sen] # 辞書にない単語にはUNKのIDを割り振ります
    return out

In [14]:
# 日本語のテキストを単語IDに変換します。
id_text = [sentence_to_ids(vocab, sen) for sen in text]

In [15]:
print(text[0])
print(id_text[0])

['私', 'は', 'その', '人', 'を', '常に', '先生', 'と', '呼ん', 'で', 'い', 'た', '。']
[10, 5, 23, 39, 9, 689, 26, 12, 485, 13, 19, 3, 4]


### 1.6. Padding
NLPではよくsequence（文や単語の列など）の長さを統一するためにPaddingという操作を行います。

各sequenceの長さが揃っていないと行列演算を行えないからです。

ここではmax_length以下の長さのsequenceにはPAD(=0)を後ろに必要なだけ追加します。

max_lengthを超える長さのsequenceはデータから省くor末尾を必要なだけ削るなどをして対処します。

In [16]:
def pad_seq(seq, max_length):
    """Paddingを行う関数

    :param seq: list of int, 単語のインデックスのリスト
    :param max_length: int, バッチ内の系列の最大長
    :return seq: list of int, 単語のインデックスのリスト
    """
    seq += [PAD for i in range(max_length - len(seq))]
    return seq

## 2. CBOW (Continuous Bag of Words)

周囲の単語($w(t-2)...w(t+2)$)からターゲットの単語($w(t)$)を予測するタスクを解くことで単語のベクトルを学習します。

まず周囲の単語を埋め込み層でベクトルにしたのち、その和をとってから全結合層を介してターゲットとなる単語を予測します。

<img src="../image/cbow.png">

<div style="text-align: center;">
出典:[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
</div>

In [17]:
# Hyper Parameters
batch_size = 64 # ミニバッチのサイズ
n_batches = 300 # 今回学習するミニバッチの数
vocab_size = len(vocab.word2id) # 語彙の総数
embedding_size = 300 # 各単語に割り当てるベクトルの次元数

### Tips: イテレータの使い方

2.1 データローダーでイテレータを使うのですが、イテレータに馴染みがない方もいるかと思われるので、ここでPythonでのイテレータの簡単な使い方に触れておきます。

In [18]:
class TestIter(object):
    def __init__(self):
        self.iter = 0
        self.max_iter = 5
    
    def __iter__(self): # 必須
        print("iter関数が呼び出されました")
        return self
    
    def __next__(self):
        self.iter += 1
        print("next関数が呼び出されました({}回目)".format(self.iter))
        if self.iter < self.max_iter:
            return None
        else:
            print("max_iterに達したので終了します")
            raise StopIteration

In [19]:
# インスタンス生成
testiter = TestIter()
# イテレータを生成
t = iter(testiter)

iter関数が呼び出されました


In [20]:
next(t)
next(t)

next関数が呼び出されました(1回目)
next関数が呼び出されました(2回目)


In [21]:
testiter = TestIter()
for i in testiter:
    pass

iter関数が呼び出されました
next関数が呼び出されました(1回目)
next関数が呼び出されました(2回目)
next関数が呼び出されました(3回目)
next関数が呼び出されました(4回目)
next関数が呼び出されました(5回目)
max_iterに達したので終了します


### 2.1. データローダー

データセットから一部のデータ（ミニバッチ）を取得するデータローダーを定義します。

データセットというのはデータ全体のことを指し、ミニバッチはデータセットを小分けにしたデータの集合を指します。
データセットはサイズが大きいケースが多いため、一度に全てのデータセットをモデルに与えて学習させるのではなく、ミニバッチという単位でデータを小分けにしてモデルに複数回与えることで学習させます。
(参考：バッチ学習、ミニバッチ学習、オンライン学習)

通常、教師あり学習では各ミニバッチは入力データと正解データから構成されます。
今回は入力データ(batch_X)が周辺単語(のID群)、正解データ(batch_Y)がターゲットの単語(のID)となります。

各ミニバッチはサイズが(batch_size, window*2)のTensorになるようにします。
この際、先ほど定義したPaddingという操作を行って各データの長さを揃えます。

In [22]:
class DataLoaderCBOW(object):
    """CBOWのデータローダー"""
    def __init__(self, text, batch_size, window=3):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語とターゲットの単語の最大距離
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.s_pointer = 0 # データセット上を走査する文単位のポインタ
        self.w_pointer = 0 # データセット上を走査する単語単位のポインタ
        self.max_s_pointer = len(text) # データセットに含まれる文の総数

    def __iter__(self):
        return self

    def __next__(self):
        batch_X = []
        batch_Y = []
        while len(batch_X) < self.batch_size:
            # 走査する対象の文
            sen = self.text[self.s_pointer]
            
            # 予測すべき単語
            word_Y = sen[self.w_pointer]
            
            # 入力となる単語群を取得
            start = max(0, self.w_pointer - self.window)
            word_X = sen[start:self.w_pointer] + \
                sen[self.w_pointer + 1:self.w_pointer + self.window + 1]
            word_X = pad_seq(word_X, self.window * 2)
            
            batch_X.append(word_X)
            batch_Y.append(word_Y)
            self.w_pointer += 1
            
            if self.w_pointer >= len(sen):
                # 文を走査し終わったら次の文の先頭にポインタを移行する
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    # 全ての文を走査し終わったら終了する
                    self.s_pointer = 0
                    raise StopIteration

        # データはtorch.Tensorにする必要があります。dtype, deviceも指定します。
        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)

        return batch_X, batch_Y

### Tips: Embedding

EmbeddingとはNLPでDeep Learningをするときにほぼ必ず使われる手法で、Embeddingを行うLayerをEmbedding Layer（埋め込み層）と言います。

端的に言うとEmbeddingとは「ある単語」を（に）「d次元のベクトル」に埋め込む（を割り当てる）ことを指します。

語彙に含まれるすべての単語に対応する行ベクトルを縦方向につなげた（語彙数 × ベクトルの次元）の2次行列がEmbedding Layer（埋め込み層）のパラメータで、Embedding Matrixとも呼ばれます。

この埋め込み層では入力が単語のID（りんご：13）、出力が入力の単語IDに対応するd次元ベクトル（Embedding Matrixの13行目のベクトル）です。

そしてWord2Vecでは学習が終わった後のEmbedding Layerのパラメータを単語表現として扱います。

* 用語まとめ
    * Embedding
        * 単語をベクトルに埋め込むこと
    * Embedding Layer
        * Embeddingを行うLayer
    * Embedding Matrix
        * Embedding Layerのパラメータで、サイズは(語彙数, ベクトルの次元)

In [ ]:
# 使用例
e = nn.Embedding(vocab_size, embedding_size) # 埋め込み層を定義

# 単語のIDからなるベクトルを入力
words_1D = torch.tensor([1,2,3], dtype=torch.long) # dtypeは必ずtorch.longにする
print("Before embedding:", words_1D.shape)
embed_words = e(words_1D) # 単語IDをベクトルに変換
print("After embedding:", embed_words.shape)
print()

# 埋め込み層への入力は2Dでも可
words_2D = torch.tensor([[1,2,3], [4,5,6]], dtype=torch.long)
print("Before embedding:", words_2D.shape)
embed_words = e(words_2D)
print("After embedding:", embed_words.shape)

### 2.2. CBOWモデル定義

目的関数はNegative Log Likelihood (NLL)です。

単語$w_t$の条件付き確率$p(w_t|w_{t-c}, ... , w_{t-1}, w_{t+1}, ..., w_{t+c})$は、周囲の単語が$w_{t-c}, ... , w_{t-1}, w_{t+1}, ..., w_{t+c}$であるときにt番目の単語が$w_t$である確率を表します。周囲の単語ベクトルの和をとってから線形変換して次元を`embedding_size`から`vocab_size`に変換した後、softmax関数を介して得られた確率分布により求められます。

\begin{align}
& {\mathcal L}_{CBOW}=-\frac{1}{T}\sum_{t=1}^T \log p(w_{t}|w_{t-c}, ... , w_{t-1}, w_{t+1}, ..., w_{t+c})\\
& =-\frac{1}{T}\sum_{t=1}^T \log ({\rm softmax}((\sum_{-c \leq j \leq c,j\neq0}\boldsymbol{v}_{t+j})^T\boldsymbol{W})_{s_t})
\end{align}

- $T$: バッチサイズ
- $w_t$: 予測すべき単語
- $c$: window幅(≧1)
- $\boldsymbol{v}_t$: $w_t$のembedding vector
- $s_t$: $w_t$の単語ID
- $\boldsymbol{W}$: (embedding_size, vocab_size)の行列

In [ ]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(CBOW, self).__init__()
        """
        :param vocab_size: int, 語彙の総数
        :param embedding_size: int, 単語埋め込みベクトルの次元
        """
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        # 埋め込み層
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        # 全結合層(バイアスなし)
        self.linear = nn.Linear(self.embedding_size, self.vocab_size, bias=False)
        
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, batch_X, batch_Y):
        """
        :param batch_X: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :param batch_Y: torch.Tensor(dtype=torch.long), (batch_size,)
        :return loss: torch.Tensor(dtype=torch.float), CBOWのloss
        """
        emb_X = self.embedding(batch_X) # (batch_size, window*2, embedding_size)
        # paddingした部分を無視するためにマスクをかけます
        emb_X = emb_X * (batch_X != PAD).float().unsqueeze(-1) # (batch_size, window*2, embedding_size)
        sum_X = torch.sum(emb_X, dim=1) # (batch_size, embedding_size)
        lin_X = self.linear(sum_X) # (batch_size, vocab_size)
        log_prob_X = F.log_softmax(lin_X, dim=-1) # (batch_size, vocab_size)
        loss = F.nll_loss(log_prob_X, batch_Y)
        return loss

### 2.3. 訓練

In [ ]:
# モデル
cbow = CBOW(vocab_size, embedding_size).to(device) # iLectで実行する場合warning (GPU is too old) が出ますが, 実行に問題はないので気にせず進めてください.
# optimizer
optimizer_cbow = optim.Adam(cbow.parameters())
# データローダー
dataloader_cbow = DataLoaderCBOW(id_text, batch_size)

In [ ]:
def compute_loss(model, inputs, optimizer=None, is_train=True):
    """lossを計算するための関数
    
    is_train=Trueならモデルをtrainモードに、
    is_train=Falseならモデルをevaluationモードに設定します
    
    :param model: 学習させるモデル
    :param inputs: モデルへの入力
    :param optimizer: optimizer
    :param is_train: bool, モデルtrainさせるか否か
    """
    model.train(is_train)

    # lossを計算します。
    loss = model(*inputs)

    if is_train:
        # .backward()を実行する前にmodelのparameterのgradientを全て0にセットします
        optimizer.zero_grad()
        # parameterのgradientを計算します。
        loss.backward()
        # parameterのgradientを用いてparameterを更新します。
        optimizer.step()

    return loss.item()

In [ ]:
start_at = time.time()

for batch_id, (batch_X, batch_Y) in enumerate(dataloader_cbow):
    loss = compute_loss(cbow, (batch_X, batch_Y), optimizer=optimizer_cbow, is_train=True)
    if batch_id % 100 == 0:
        print("batch:{}, loss:{:.4f}".format(batch_id, loss))
    if batch_id >= n_batches:
        break

end_at = time.time()

print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

### 2.4. 訓練済みパラメータの保存

CBOWの学習が終わった時の埋め込み層のパラメータが、各単語の特徴を表現していると捉えます。

そのためCBOWの埋め込み層のパラメータのみを保存します。

In [ ]:
# 埋め込み層のパラメータのみを保存する
torch.save(cbow.embedding.weight.data.cpu().numpy(),  "../data/cbow_embedding_{}b.pth".format(n_batches))

In [ ]:
# 保存したパラメータの読み込み方
e = torch.load("../data/cbow_embedding_{}b.pth".format(n_batches))
e

## 3. Skipgram

ターゲットの単語($w(t)$)から周囲の単語($w(t-2)...w(t+2)$)を予測するタスクを解くことで単語のベクトルを学習します。

<img src="../image/skipgram.png">

<div style="text-align: center;">
出典:[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
</div>

### 3.1. データローダーの定義

In [ ]:
class DataLoaderSG(object):
    """Skipgramのためのデータローダー"""
    def __init__(self, text, batch_size, window=3):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語と入力単語の最大距離
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.s_pointer = 0 # データセット上を走査する文単位のポインタ
        self.w_pointer = 0 # データセット上を走査する単語単位のポインタ
        self.max_s_pointer = len(text) # データセットに含まれる文の総数

    def __iter__(self):
        return self

    def __next__(self):
        batch_X = []
        batch_Y = []

        while len(batch_X) < self.batch_size:
            sen = self.text[self.s_pointer]
            
            # Skipgramでは入力が1単語
            word_X = # WRITE ME

            # 出力は周辺単語
            start = # WRITE ME
            word_Y = # WRITE ME
            word_Y = # WRITE ME, paddingが必要

            batch_X.append(word_X)
            batch_Y.append(word_Y)
            self.w_pointer += 1

            if self.w_pointer >= len(sen):
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    self.s_pointer = 0
                    raise StopIteration

        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)

        return batch_X, batch_Y

### Tips: torch.gather(input, dim, index)
指定したdimに沿って、以下のような操作を行う。(inputとindexが3D tensorだった場合)
\begin{align}
& out[ i ][ j ][ k ] = input[index[i][j][k]][j][k],~ if dim == 0\\
& out[ i ][ j ][ k ] = input[i][index[i][j][k]][k],~ if dim == 1\\
& out[ i ][ j ][ k ] = input[i][j][index[i][j][k]],~ if dim == 2\\
\end{align}

詳しくはchap1を参照してください。

※参考:https://pytorch.org/docs/stable/torch.html#torch.gather

### 3.2. モデルの定義

目的関数はNegative Log Likelihood (NLL)です。

$p(w_{t+j}|w_t)$は、単語$w_t$の周囲に単語$w_{t+j}$が存在する確率を表します。この確率は単語$w_t$のembedding vectorを線形変換により次元を`embedding_size`から`vocab_size`に変換した後、softmax関数を介して得られた確率分布によって求められます。（下の数式の3行目から4行目に相当）

実装する際は、各jについて同じ計算$\log({\rm softmax}(\boldsymbol{v}_{t}^T\boldsymbol{W})_{s_{t+j}})$をするのは非効率なので、$\log({\rm softmax}(\boldsymbol{v}_{t}^T\boldsymbol{W}))$を計算した後にすべてのjに対応する値をtorch.gatherを使って抽出できると計算時間を短縮できます。

\begin{align}
& {\mathcal L}_{SG}=-\frac{1}{T}\sum_{t=1}^T \log p(w_{t-c}, ... , w_{t-1}, w_{t+1}, ..., w_{t+c}|w_{t})\\
& = -\frac{1}{T}\sum_{t=1}^T \log \prod_{-c \leq j \leq c,j\neq0} p(w_{t+j}|w_t)\\
& =-\frac{1}{T}\sum_{t=1}^T \sum_{-c \leq j \leq c,j\neq0} \log p(w_{t+j}|w_t)\\
& =-\frac{1}{T}\sum_{t=1}^T \sum_{-c \leq j \leq c,j\neq0} \log ({\rm softmax}(\boldsymbol{v}_{t}^T\boldsymbol{W})_{s_{t+j}})
\end{align}

- $T$: バッチサイズ
- $w_t$: 入力単語
- $c$: window幅(≧1)
- $\boldsymbol{v}_t$: $w_t$のembedding vector
- $s_t$: $w_t$の単語ID
- $\boldsymbol{W}$: (embedding_size, vocab_size)の行列

In [ ]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        """
        :param vocab_size: int, 語彙の総数
        :param embedding_size: int, 単語埋め込みベクトルの次元
        """
        super(Skipgram, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        self.embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        self.linear = nn.Linear(self.embedding_size, self.vocab_size, bias=False)

    def forward(self, batch_X, batch_Y):
        """
        :param batch_X: torch.Tensor(dtype=torch.long), (batch_size,)
        :param batch_Y: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :return loss: torch.Tensor(dtype=torch.float), Skipgramのloss
        """
        emb_X = # WRITE ME (batch_size, embedding_size)
        lin_X = # WRITE ME (batch_size, vocab_size)
        log_prob_X = # WRITE ME  (batch_size, vocab_size)、各単語の確率
        log_prob_X = torch.gather(log_prob_X, 1, batch_Y) # (batch_size, window*2)
        # paddingした単語のlossは計算しないようにマスクをかけます(=lossの該当部分を0にします)
        log_prob_X = log_prob_X * (batch_Y != PAD).float() # (batch_size, window*2)
        loss = log_prob_X.sum(1).mean().neg()
        return loss

### 3.3. 訓練

In [ ]:
sg = Skipgram(vocab_size, embedding_size).to(device)
optimizer_sg = optim.Adam(sg.parameters())
dataloader_sg = DataLoaderSG(id_text, batch_size)

In [ ]:
start_at = time.time()
for batch_id, (batch_X, batch_Y) in enumerate(dataloader_sg):
    loss = compute_loss(sg, (batch_X, batch_Y), optimizer=optimizer_sg, is_train=True)
    if batch_id % 100 == 0:
        print("batch:{}, loss:{:.4f}".format(batch_id, loss))
    if batch_id >= n_batches:
        break
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

### 3.4. 訓練済みパラメータの保存

In [ ]:
# 埋め込み層のパラメータのみを保存する
torch.save(sg.embedding.weight.data.cpu().numpy(), "../data/sg_embedding_{}b.pth".format(n_batches))

## 4. Skipgram with Negative Sampling

### Tips: Negative Sampling


[Mikolov et al. 2013]によって提案された単語ベクトルの学習するために用いる方法の1つで、gensimのword2vecにも実装されています。
Negative Samplingとは実際には存在しないデータ(=負例)をランダムにサンプリングすることです。

提案された背景として、従来のSkipgramやCBOWでは文中におけるターゲットの単語($w_t$)とその周辺単語($w_{t+j}$)が共起する事象の尤度を最大化して単語ベクトルを学習させていました。ところがこの尤度ではsoftmax関数を使うため計算量が語彙数$n$(通常$10^5$から$10^7$ほど)に比例するので$O(n)$であり膨大でした。

そこでsoftmax関数の代わりにsigmoid関数を使うことによって計算量を$O(1)$に抑える代わりに、単語ベクトルの質を担保するために新たにNegative Samplingを導入しました。
元の論文では、実際に共起する単語ペア、すなわち正例の尤度を最大化し、同時にNegative Samplingによって得られた負例の尤度を最小化しています。

参考:[Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)

### 4.0. Negative Sampling準備
元論文では、負例をサンプリングする際に使う確率分布$P_n(w)$をコーパスにおける単語$w$の出現回数$U(w)$から以下のように定義しています。

\begin{align}
P_n(w) = \frac{U(w)^{3/4}}{\sum_{w \in W}U(w)^{3/4}}
\end{align}

この確率分布の求め方はヒューリスティックですが元論文では試行錯誤の結果これが良かったと報告されています。恐らく3/4には高頻度語(is,theなど)ばかりがサンプリングされるのを防ぐ効果があったためだと思われます。

実際に実装する際は、特殊なトークンであるPADやUNKはコーパスに登場しないことに注意してください。

In [ ]:
# negative samplingに使う確率分布
weights = np.power([0, 0] + list(vocab.raw_vocab.values()), 0.75)
weights = weights / weights.sum()

### 4.1. データローダー
3.Skipgramとは異なりデータローダーの中でNegative Samplingを行います。

In [ ]:
class DataLoaderSGNS(object):
    def __init__(self, text, batch_size, window=3, n_negative=5, weights=None):
        """
        :param text: list of list of int, 単語をIDに変換したデータセット
        :param batch_size: int, ミニバッチのサイズ
        :param window: int, 周辺単語と入力単語の最大距離
        :param n_negative: int, 負例の数
        :param weights: numpy.ndarray, Negative Samplingで使う確率分布
        """
        self.text = text
        self.batch_size = batch_size
        self.window = window
        self.n_negative = n_negative
        self.weights = None
        if weights is not None:
            self.weights = torch.FloatTensor(weights) # negative samplingに使う確率分布
        self.s_pointer = 0 # 文のポインタ
        self.w_pointer = 0 # 単語のポインタ
        self.max_s_pointer = len(text)

    def __iter__(self):
        return self
    
    def __next__(self):
        batch_X = []
        batch_Y = []
        batch_N = [] # 負例
        while len(batch_X) < self.batch_size:
            sen = self.text[self.s_pointer]
            start = max(0, self.w_pointer - self.window)
            word_X = sen[self.w_pointer]
            word_Y = sen[start:self.w_pointer] + \
                sen[self.w_pointer + 1:self.w_pointer + self.window + 1]
            word_Y = pad_seq(word_Y, self.window * 2)
            batch_X.append(word_X)
            batch_Y.append(word_Y)

            # 多項分布で負例をサンプリング
            # 実装を簡略化するために、正例の除去は行っていません
            negative_samples = torch.multinomial(self.weights, self.n_negative) # (n_negative,)
            batch_N.append(negative_samples.unsqueeze(0)) # (1, n_negative)

            self.w_pointer += 1
            if self.w_pointer >= len(sen):
                self.w_pointer = 0
                self.s_pointer += 1
                if self.s_pointer >= self.max_s_pointer:
                    self.s_pointer = 0
                    raise StopIteration

        batch_X = torch.tensor(batch_X, dtype=torch.long, device=device)
        batch_Y = torch.tensor(batch_Y, dtype=torch.long, device=device)
        batch_N = torch.cat(batch_N, dim=0).to(device) # (batch_size, n_negative)

        return batch_X, batch_Y, batch_N

### 4.2. モデルの定義

Skipgram with Negative Sampling (SGNS)の目的関数では、$p(w_{t+j}|w_t)=\sigma (\boldsymbol{v}'^T_{t+j}\boldsymbol{v}_t)$と定義することで計算量を削減しています。活性化関数にsoftmax関数ではなくsigmoid関数を用いているためです。

正例の尤度の最大化に加えてnegative samplingをして負例の尤度を最小化するため目的関数は以下のように記述されます。

\begin{align}
& {\mathcal L}_{SGNS}= \frac{1}{T} \sum_{t=1}^T NLL(w_t) \\
& =-\frac{1}{T} \sum_{t=1}^T \log \left( \prod_{-c \leq j \leq c,j\neq0} p(w_{t+j}|w_t) \prod_{w_n \in S} \left( 1 - p(w_n|w_t) \right)  \right) \\
& =-\frac{1}{T} \sum_{t=1}^T \left( \sum_{-c \leq j \leq c,j\neq0} \log p(w_{t+j}|w_t) + \sum_{w_n \in S} \log (1 - p(w_n|w_t)) \right) \\
& =-\frac{1}{T} \sum_{t=1}^T \left( \sum_{-c \leq j \leq c,j\neq0} \log \sigma (\boldsymbol{v}'^T_{t+j} \boldsymbol{v}_t) + \sum_{w_n \in S} \log (1 - \sigma (\boldsymbol{v}'^T_n\boldsymbol{v}_t)) \right) \\
& =-\frac{1}{T} \sum_{t=1}^T \left( \sum_{-c \leq j \leq c,j\neq0} \log \sigma (\boldsymbol{v}'^T_{t+j}\boldsymbol{v}_t) + \sum_{w_n \in S} \log \sigma (-\boldsymbol{v}'^T_n\boldsymbol{v}_t) \right) \\
\end{align}

- $T$: バッチサイズ
- $w_t$: 入力単語
- $c$: window幅(≧1)
- $S$: ランダムに選んだNegative Sampleの集合
- $\boldsymbol{v}_t$: 入力単語$w_t$のembedding vector
- $\boldsymbol{v}'_t$: 出力単語$w_t$のembedding vector
- $\sigma$: シグモイド関数

In [ ]:
class SGNS(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        """
        :param vocab_size: int, 語彙の総数
        :param embedding_size: int, 単語埋め込みベクトルの次元
        """
        super(SGNS, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        # 入力単語の埋め込み層
        self.i_embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        # 出力単語の埋め込み層
        self.o_embedding = nn.Embedding(self.vocab_size, self.embedding_size)
        
        nn.init.xavier_uniform_(self.i_embedding.weight)
        nn.init.xavier_uniform_(self.o_embedding.weight)

    def forward(self, batch_X, batch_Y, batch_N):
        """
        :param batch_x: torch.Tensor(dtype=torch.long), (batch_size,)
        :param batch_y: torch.Tensor(dtype=torch.long), (batch_size, window*2)
        :param batch_n: torch.Tensor(dtype=torch.long), (batch_size, n_negative)
        """
        embed_X = self.i_embedding(batch_X).unsqueeze(2) # (batch_size, embedding_size, 1)
        embed_Y = self.o_embedding(batch_Y) # (batch_size, window*2, embedding_size)
        embed_N = self.o_embedding(batch_N).neg() # (batch_size, n_negative, embedding_size)
        loss_Y = torch.bmm(embed_Y, embed_X).squeeze().sigmoid().log() # (batch_size, window*2)
        loss_Y = loss_Y * (batch_Y != PAD).float() # (batch_size, window*2)
        loss_Y = loss_Y.sum(1) # (batch_size,)
        loss_N = torch.bmm(embed_N, embed_X).squeeze().sigmoid().log().sum(1) # (batch_size,)
        return -(loss_Y + loss_N).mean()

### 4.3. 訓練

In [ ]:
sgns = SGNS(vocab_size, embedding_size).to(device)
optimizer_sgns = optim.Adam(sgns.parameters())
dataloader_sgns = DataLoaderSGNS(id_text, batch_size, n_negative=5, weights=weights)
start_at = time.time()
for batch_id, (batch_X, batch_Y, batch_N) in enumerate(dataloader_sgns):
    loss = compute_loss(sgns, (batch_X, batch_Y, batch_N), optimizer=optimizer_sgns, is_train=True)
    if batch_id % 100 == 0:
        print("batch:{}, loss:{:.4f}".format(batch_id, loss))
    if batch_id >= n_batches:
        break
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

### 4.4. 保存

In [ ]:
# Embeddingのパラメータのみを保存する
torch.save(sgns.i_embedding.weight.data.cpu().numpy(), "../data/sgns_embedding_{}b.pth".format(n_batches))

## 5. Word Similarity

単語の意味が似ていれば単語ベクトルも似たようなものになっているはずです。

単語同士が"似ている"かどうかを判断するために、しばしば単語ベクトル同士のcos類似度が用いられます。

cos類似度は単語ベクトル同士のなす角$\theta$を用いて、

\begin{align}
 cos\theta=\frac{\boldsymbol{u}^T\boldsymbol{v}}{|\boldsymbol{u}||\boldsymbol{v}|}
\end{align}

のように定義されます。単語同士の意味が似ているほどcos類似度は1に近くなると考えられています。

Word2Vecで学習すると、どの単語とどの単語が似ていると判断されるのか見てみましょう。

In [ ]:
def compute_word_similarity(embedding_path, word, n):
    """
    与えられた単語に最も似ている単語とcos類似度を返す関数

    :param embedding_path: str, 保存した埋め込み層のパラメータのパス
    :param word: str, 単語
    :param n: int
    :return out: str, 上位n個の類似単語とそのcos類似度
    """
    embedding = torch.load(embedding_path)

    # 単語ベクトルを全て単位ベクトルにする
    norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    embedding /= norm
    e = embedding[vocab.word2id[word]]

    # 単語ベクトル同士のcos類似度を計算する
    cos_sim = np.dot(embedding, e.reshape(-1, 1)).reshape(-1,)
    most_sim = np.argsort(cos_sim)[::-1][1:n+1] # 自分は除く
    most_sim_words = [vocab.id2word[_id] for _id in most_sim]
    top_cos_sim = cos_sim[most_sim]
    out = ", ".join([w+"({:.4f})".format(v) for w, v in zip(most_sim_words, top_cos_sim)])
    return out

In [ ]:
# 300バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_similarity(
        "../data/" + model + "_embedding_300b.pth", "私", 5))

In [ ]:
# 600バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_similarity(
        "../data/" + model + "_embedding_600b.pth", "私", 5))

In [ ]:
# 900バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_similarity(
        "../data/" + model + "_embedding_900b.pth", "私", 5))

## 6. Word Analogy

次は単語ベクトルの足し算・引き算によって単語を類推できるかどうかによってモデルを評価します。

ここでは$v($"父"$) - v($"男"$) + v($"女"$)$とcos類似度で測ったときに最も似ている単語を列挙します。

より"母"が上位に来ているモデルが優れていると判断できます。

In [ ]:
def compute_word_analogy(embedding_path, word1, word2, word3, n):
    """word1 - word2 + word3と最も類似度の高い単語を返す関数

    :param embedding_path: str, 保存した埋め込み層のパラメータのパス
    :param word1: str, 単語
    :param word2: str, 単語
    :param word3: str, 単語
    :param n: int
    :return out: str, 上位n個の類似単語とそのcos類似度
    """
    embedding = torch.load(embedding_path)
    w1 = vocab.word2id[word1]
    w2 = vocab.word2id[word2]
    w3 = vocab.word2id[word3]
    v1 = embedding[w1]
    v2 = embedding[w2]
    v3 = embedding[w3]
    v = v1 - v2 + v3
    v /= np.linalg.norm(v, ord=2)
    
    norm = np.linalg.norm(embedding, ord=2, axis=1, keepdims=True)
    norm = np.where(norm==0, 1, norm) # 0で割ることを避ける
    embedding /= norm
    
    cos_sim = np.dot(embedding, v.reshape(-1, 1)).reshape(-1,)
    most_sim = np.argsort(cos_sim)[::-1][0:n]
    most_sim_words = [vocab.id2word[_id] for _id in most_sim]
    top_cos_sim = cos_sim[most_sim]
    out = ", ".join([w+"({:.4f})".format(v) for w, v in zip(most_sim_words, top_cos_sim)])
    return out

In [ ]:
# 300バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_analogy(
        "../data/" + model + "_embedding_300b.pth", "父", "男", "女", 5))

In [ ]:
# 600バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_analogy(
        "../data/" + model + "_embedding_600b.pth", "父", "男", "女", 5))

In [ ]:
# 900バッチだけ学習した時
models = ["cbow", "sg", "sgns"]
for model in models:
    print(model+"\t:", compute_word_analogy(
        "../data/" + model + "_embedding_900b.pth", "父", "男", "女", 5))

## 参考文献

* 元論文
    * [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
    * [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* [スタンフォード大学のDeep Learning for NLP講座の講義ノート](https://cs224d.stanford.edu/lecture_notes/notes1.pdf)
* [gensimでのWord2Vec実装](https://radimrehurek.com/gensim/models/word2vec.html)